<a href="https://colab.research.google.com/github/sukilsiva/Machine-Learnig-Deep-Learning-Code/blob/master/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Embedding, Dense, GRU, Flatten
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import cdist

In [2]:
data = pd.read_csv("/content/AI-Sentiment-Analysis-on-IMDB-Dataset/imdb_tr.csv",sep = ',', encoding = 'latin')
data.head()

,row_Number,text,polarity
0,2148,"first think another Disney movie, might good, ...",1
1,23577,"Put aside Dr. House repeat missed, Desperate H...",0
2,1319,"big fan Stephen King's work, film made even gr...",1
3,13358,watched horrid thing TV. Needless say one movi...,0
4,9495,truly enjoyed film. acting terrific plot. Jeff...,1


In [3]:
data.isna().sum()

row_Number    0
text          0
polarity      0
dtype: int64

In [4]:
data.duplicated().sum()

0

In [0]:
x = data['text']
y = data['polarity']

In [6]:
x

0        first think another Disney movie, might good, ...
1        Put aside Dr. House repeat missed, Desperate H...
2        big fan Stephen King's work, film made even gr...
3        watched horrid thing TV. Needless say one movi...
4        truly enjoyed film. acting terrific plot. Jeff...
                               ...                        
24995    kid 50's 60's anything connected Disney defini...
24996    course reading review seen film already. 'Raja...
24997    read "There's Girl Soup" came Peter Sellers's ...
24998    film quite boring. snippets naked flesh tossed...
24999    Although film somewhat filled eighties cheese ...
Name: text, Length: 25000, dtype: object

In [60]:
x.shape

(25000,)

In [7]:
y

0        1
1        0
2        1
3        0
4        1
        ..
24995    0
24996    1
24997    0
24998    0
24999    1
Name: polarity, Length: 25000, dtype: int64

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

**TOKENZIE DATA**

In [0]:
num_words = 1000

In [0]:
tokenizer = Tokenizer(num_words = num_words)

In [0]:
tokenizer.fit_on_texts(x)

from below word_index we can able to understand what token assigned for which letter

In [12]:
tokenizer.word_index

{'br': 1,
 'movie': 2,
 'film': 3,
 'one': 4,
 'like': 5,
 "it's": 6,
 'good': 7,
 'the': 8,
 'it': 9,
 'time': 10,
 'even': 11,
 'would': 12,
 'story': 13,
 'really': 14,
 'see': 15,
 'well': 16,
 'much': 17,
 'bad': 18,
 'get': 19,
 'also': 20,
 'people': 21,
 'first': 22,
 'great': 23,
 "don't": 24,
 'made': 25,
 'way': 26,
 'make': 27,
 'could': 28,
 'movies': 29,
 'think': 30,
 'characters': 31,
 'watch': 32,
 'two': 33,
 'films': 34,
 'character': 35,
 'seen': 36,
 'many': 37,
 'i': 38,
 'life': 39,
 'plot': 40,
 'never': 41,
 'acting': 42,
 'little': 43,
 'best': 44,
 'love': 45,
 'this': 46,
 'show': 47,
 'know': 48,
 'ever': 49,
 'better': 50,
 'end': 51,
 'still': 52,
 'man': 53,
 'and': 54,
 'say': 55,
 'scene': 56,
 'scenes': 57,
 'go': 58,
 'something': 59,
 'back': 60,
 "i'm": 61,
 'real': 62,
 'watching': 63,
 'though': 64,
 "doesn't": 65,
 'years': 66,
 'old': 67,
 'thing': 68,
 'actors': 69,
 'work': 70,
 '10': 71,
 'another': 72,
 "didn't": 73,
 'new': 74,
 'funny': 7

In [65]:
x_train_tokens = tokenizer.texts_to_sequences(x_train)
x_train_tokens[1]

[2,
 97,
 217,
 100,
 111,
 749,
 24,
 32,
 21,
 24,
 90,
 32,
 262,
 79,
 21,
 523,
 577,
 459,
 340,
 101,
 199,
 13,
 14,
 14,
 882,
 144,
 887,
 485,
 21,
 52,
 146,
 43,
 24,
 85,
 2,
 558,
 460,
 818,
 786,
 752,
 2,
 30,
 827,
 27,
 69]

In [0]:
x_test_tokens = tokenizer.texts_to_sequences(x_test)

PADDING AND TRUNCATING

The Recurrent Neural Network can take sequences of arbitrary length as input, but in order to use a whole batch of data, the sequences need to have the same length. There are two ways of achieving this: (A) Either we ensure that all sequences in the entire data-set have the same length, or (B) we write a custom data-generator that ensures the sequences have the same length within each batch.

Solution (A) is simpler but if we use the length of the longest sequence in the data-set, then we are wasting a lot of memory. This is particularly important for larger data-sets.

So in order to make a compromise, we will use a sequence-length that covers most sequences in the data-set, and we will then truncate longer sequences and pad shorter sequences.

First we count the number of tokens in all the sequences in the data-set.



In [17]:
num_tokens = [len(tokens) for tokens in x_train_tokens + y_train_tokens]
num_tokens = np.array(num_tokens)

ERROR! Session/line number was not unique in database. History logging moved to new session 65


In [18]:
np.mean(num_tokens)

79.3444

In [19]:
np.max(num_tokens)

588

In [20]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

190


When padding or truncating the sequences that have a different length, we need to determine if we want to do this padding or truncating 'pre' or 'post'. If a sequence is truncated, it means that a part of the sequence is simply thrown away. If a sequence is padded, it means that zeros are added to the sequence.

So the choice of 'pre' or 'post' can be important because it determines whether we throw away the first or last part of a sequence when truncating, and it determines whether we add zeros to the beginning or end of the sequence when padding. This may confuse the Recurrent Neural Network

In [0]:
pad = 'pre'

In [72]:
x_train_pad = pad_sequences(x_train_tokens, maxlen = max_tokens , padding = pad, truncating = pad)
x_train_pad[1]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   2,  97, 217, 100, 111, 749,  24,  32,  21,  24,  90,
        32, 262,  79,  21, 523, 577, 459, 340, 101, 199,  13,  14,  14,
       882, 144, 887, 485,  21,  52, 146,  43,  24,  85,   2, 55

In [0]:
x_test_pad = pad_sequences(x_test_tokens, maxlen = max_tokens, padding = pad, truncating = pad)

INVERSE FROM TOKENS TO TEXT

this is done if needed

In [0]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

In [0]:
def tokens_to_text(tokens):
  word = [inverse_map[token] for token in tokens if token != 0]
  text = " ".join(word)
  return text

In [30]:
tokens_to_text(x_train_tokens[1])

"movie got 1 can't give weak don't watch people don't want watch either makes people happen cool parts case fact true story really really creepy guy worked killed people still feel little don't every movie scary self t rent buy movie think 20 make actors"

RNN

In [0]:
model = Sequential()

In [0]:
embedding_size = 8

In [0]:
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='layer_embedding'))

In [0]:
model.add(GRU(units = 16, return_sequences = True))

In [0]:
model.add(GRU(units = 8, return_sequences = True))

In [0]:
model.add(GRU(units = 8, return_sequences= True))

In [0]:
model.add(Flatten())

In [0]:
model.add(Dense(1, activation = 'sigmoid'))

In [0]:
model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [109]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 190, 8)            8000      
_________________________________________________________________
gru_1 (GRU)                  (None, 190, 16)           1200      
_________________________________________________________________
gru_2 (GRU)                  (None, 190, 8)            600       
_________________________________________________________________
gru_3 (GRU)                  (None, 190, 8)            408       
_________________________________________________________________
dense_1 (Dense)              (None, 190, 1)            9         
_________________________________________________________________
dense_2 (Dense)              (None, 190, 16)           32        
_________________________________________________________________
dense_3 (Dense)              (None, 190, 32)          

In [110]:
x_train_pad.shape

(20000, 190)

In [111]:
y_train.shape

(20000,)

In [0]:
model.fit(x_train_pad, y_train,
          validation_split=0.05, epochs=1,steps_per_epoch = num_words, validation_steps = num_words)

Train on 19000 samples, validate on 1000 samples
Epoch 1/1
  61/1000 [>.............................] - ETA: 25:22 - loss: 0.6932 - accuracy: 0.5009